### Create your own element, in this case a radar plot
This example is based on code from https://stackoverflow.com/questions/46564099/what-are-the-steps-to-create-a-radar-chart-in-bokeh-python
<img src='./radar_plot.png'>

In [14]:
import numpy as np
from bokeh.plotting import figure, output_notebook, show
from bokeh.models   import ColumnDataSource, LabelSet, HoverTool
output_notebook()

Loading BokehJS ...

### 1. Data for one particular plot

In [15]:
values     = [.5, 1, .8, .3, .8, .8, .8, .9]
dimensions = ['Dim 1','Dim 2','Dim 3','Dim 4','Dim 5','Dim 6','Dim 7','Dim 8']
dim_descr  = {'Dim 1': 'What was the modality of the task',
              'Dim 2': 'Was was the difficulty of the task',
              'Dim 3': 'What was the valence of the task',
              'Dim 4': 'How interesting was the task',
              'Dim 5': 'Was the task too repetitive',
              'Dim 6': 'Would you change the timing of the task',
              'Dim 7': 'Would you recommend the task to your colleagues?',
              'Dim 8': 'Were you aware of body motion associated with the task'}
responses  = ['Visual', 'Very difficult', 'Very positive', 'Extremely Interesting', 'Stongly Agree', 'Strongly Disagree', 'Strongly Agree','Yes']

### 2. Create Figure and Draw Outer Circle

In [16]:
p = figure(match_aspect=True)
# Draw Outter Circle
centre  = 0.5
p.circle(x=centre,y=centre,radius=0.5, fill_color=None, line_color='black', line_alpha=0.5)
show(p)

### 3. Draw intermediate circles

In [17]:
#Draw intermediate circles
p.circle(x=0.5,y=0.5,radius=.5, line_color='black', fill_color=None, line_alpha=0.5)
p.circle(x=0.5,y=0.5,radius=.1, line_color='black', fill_color=None, line_alpha=0.5, line_dash='dashed')
p.circle(x=0.5,y=0.5,radius=.2, line_color='black', fill_color=None, line_alpha=0.5, line_dash='dashed')
p.circle(x=0.5,y=0.5,radius=.3, line_color='black', fill_color=None, line_alpha=0.5, line_dash='dashed')
p.circle(x=0.5,y=0.5,radius=.4, line_color='black', fill_color=None, line_alpha=0.5, line_dash='dashed')
show(p)

### 4. Remove Grid and Non-polar Axes

In [18]:
# Visual Aspects
p.xgrid.visible=False
p.ygrid.visible=False
p.xaxis.visible=False
p.yaxis.visible=False
p.toolbar.logo=None
p.toolbar_location='below'
show(p)

### 5. Draw Polar Axes

In [19]:
def unit_poly_verts(theta, centre):
    """Return vertices of polygon for subplot axes.
    This polygon is circumscribed by a unit circle centered at (0.5, 0.5)
    """
    x0, y0, r = [centre] * 3
    verts = [(r*np.cos(t) + x0, r*np.sin(t) + y0) for t in theta]
    return verts

In [ ]:
# Obtain the Number of Dimensions
# ===============================
num_vars = len(values)
# Get the angle for each axes representing a dimension (from 0 to 2pi) + pi/2 --> To start on the y-axis
# ======================================================================================================
theta    = np.linspace(0, 2*np.pi, num_vars, endpoint=False)
theta   += np.pi/2
# Compute the intersection points with the outter circel for each of the axes
# ===========================================================================
verts    = unit_poly_verts(theta, centre)
x        = [v[0] for v in verts] 
y        = [v[1] for v in verts]

In [20]:
# Draw concentrical lines
# =======================
for i,j in zip(x,y):
    p.line(x=[centre,i],y=[centre,j], line_color='black', line_dash='dashed', line_alpha=0.5)
show(p)

### 6. Add Labels and hovering capabilities to axes

In [21]:
# Draw Outter Dots
# ================
out_dots_TOOLTIPS = [("Dim", "@desc")]
out_dots_src = ColumnDataSource({'x':x,'y':y,'desc':list(dim_descr.values())})
g_out_dots = p.circle(x='x',y='y', color='black', source=out_dots_src)
out_dots_hover = HoverTool(renderers=[g_out_dots], tooltips=out_dots_TOOLTIPS)
p.add_tools(out_dots_hover)
show(p)

In [22]:
# Draw Dimension Labels
# =====================
labels_src = ColumnDataSource({'x':[i if i >= 0.5 else i-.05 for i in x],'y':[i if i >= 0.5 else i-.05 for i in y],'text':dimensions})
labels     = LabelSet(x="x",y="y",text="text",source=labels_src)
p.add_layout(labels)
show(p)

### 7. Add Patch for a given set of data

In [23]:
def radar_patch(r, theta, centre ):
    """ Returns the x and y coordinates corresponding to the magnitudes of 
    each variable displayed in the radar plot
    """
    # offset from centre of circle
    offset = 0.0
    yt = (r*centre + offset) * np.sin(theta) + centre 
    xt = (r*centre + offset) * np.cos(theta) + centre 
    return xt, yt

In [24]:
# Compute the polar coordinates for the available data
# ====================================================
xt, yt = radar_patch(np.array(values), theta, centre)
# Use Bokeh Patch Element to draw the data
# ========================================
p.patch(x=xt, y=yt, fill_alpha=0.3, fill_color='blue', line_color='blue', line_width=2)
show(p)

In [25]:
# Patch hovering
patch_dots_TOOLTIPS = [("Response:","@desc")]
patch_dots_src = ColumnDataSource({'xt':xt,'yt':yt,'desc':responses})
patch_dots = p.circle(x='xt',y='yt',color='black', source=patch_dots_src)
patch_dots_hover = HoverTool(renderers=[patch_dots], tooltips=patch_dots_TOOLTIPS)
p.add_tools(patch_dots_hover)
show(p)

In [ ]:
def generate_radar_chart_from_vals(vals, strs, QD, color='black'):
    centre   = 0.5
    num_vars = len(vals)
    theta    = np.linspace(0, 2*np.pi, num_vars, endpoint=False)
    theta   += np.pi/2
    verts    = unit_poly_verts(theta, centre)
    x        = [v[0] for v in verts] 
    y        = [v[1] for v in verts]

    p =figure(match_aspect=True)
    # Draw Outter Dots
    out_dots_TOOLTIPS = [("Q:", "@desc")]
    out_dots_src = ColumnDataSource({'x':x,'y':y,'desc':list(QD.values())})
    g_out_dots = p.circle(x='x',y='y', color='black', source=out_dots_src)
    out_dots_hover = HoverTool(renderers=[g_out_dots], tooltips=out_dots_TOOLTIPS)
    p.add_tools(out_dots_hover)

    # Draw Outter Circle
    p.circle(x=0.5,y=0.5,radius=0.5, fill_color=None, line_color='black', line_alpha=0.5)

    # Draw concentrical lines
    for i,j in zip(x,y):
        p.line(x=[centre,i],y=[centre,j], line_color='black', line_dash='dashed', line_alpha=0.5)

    #Draw intermediate circles
    p.circle(x=0.5,y=0.5,radius=.5, line_color='black', fill_color=None, line_alpha=0.5)
    p.circle(x=0.5,y=0.5,radius=.1, line_color='black', fill_color=None, line_alpha=0.5, line_dash='dashed')
    p.circle(x=0.5,y=0.5,radius=.2, line_color='black', fill_color=None, line_alpha=0.5, line_dash='dashed')
    p.circle(x=0.5,y=0.5,radius=.3, line_color='black', fill_color=None, line_alpha=0.5, line_dash='dashed')
    p.circle(x=0.5,y=0.5,radius=.4, line_color='black', fill_color=None, line_alpha=0.5, line_dash='dashed')

    # Visual Aspects
    p.xgrid.visible=False
    p.ygrid.visible=False
    p.xaxis.visible=False
    p.yaxis.visible=False
    p.toolbar.logo=None
    p.toolbar_location='below'

    # Draw Question IDs
    labels_txt = ['Q'+str(i).zfill(2) for i in range(1,num_vars+1)]
    labels_src = ColumnDataSource({'x':[i if i >= 0.5 else i-.05 for i in x],'y':[i if i >= 0.5 else i-.05 for i in y],'text':labels_txt})
    labels     = LabelSet(x="x",y="y",text="text",source=labels_src)
    p.add_layout(labels)
    xt, yt = radar_patch(np.array(vals), theta, centre)
    p.patch(x=xt, y=yt, fill_alpha=0.3, fill_color=color, line_color=color, line_width=2)
    
    # Patch hovering
    patch_dots_TOOLTIPS = [("Response:","@desc")]
    patch_dots_src = ColumnDataSource({'xt':xt,'yt':yt,'desc':strs})
    patch_dots = p.circle(x='xt',y='yt',color='black', source=patch_dots_src)
    patch_dots_hover = HoverTool(renderers=[patch_dots], tooltips=patch_dots_TOOLTIPS)
    p.add_tools(patch_dots_hover)
    p.width=425
    p.height=425
    return p